This incomplete notebook is meant to quiery NSIDC for granules available within a polygon and at the specifed dates, cycle, etc.

The issue right now is reading in polygons from the .geojson format

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [14]:
#####################################################################################
#################################### Begin user input ###############################

# Credentials

uid = 'ccroberts'  # Enter Earthdata Login user name
pswd = 'Stratele1.' # Enter Earthdata Login password
email = 'ccroberts@ucsd.edu' # Enter Earthdata login email 

# Data product (e.g., 'ATL03')
short_name = 'ATL06'

# Identify region of interest (.geojson polygon)
site = "Ross1"
# Identify polygon file
polyPath = f"../shapes/{site}.geojson"

# Subset by date?
dateFlag = False
if dateFlag:
    # Dates in 'yyyy-mm-dd'
    # Times in 'HH:mm:ss'
    start_date = '2021-07-10'
    start_time = '00:00:00'
    end_date = '2022-08-11'
    end_time = '23:59:59'
    
# By cycle?
cycle = 16

# REVIEW THESE VALUES
# Cloud cover
#cloud_cover = [0.2, 70]

# Subsetting by bounding box, based on the area of interest inputted above
ss = 'n'
# Subsetting by geospatial file (Esri Shapefile, KML, etc.)
ps = 'y'
# Subsetting by time, based on the temporal range inputted above
ts = 'y'
# Reformatting option (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.
reformat = 'n'
# If yes, more options are needed (see code below)

# Variable subsetting (remove spaces and retain all forward slashes)
#coverage='/gt1l/land_ice_segments/h_li,/gt1l/land_ice_segments/longitude,/gt1l/land_ice_segments/latitude'
coverage=''

# Outfile location
path = ''


#################################### End user input ##################################
######################################################################################

In [49]:
with open(polyPath) as json_file:
    polygon = json.load(json_file)

# If polygon file input:
search_params = {
'short_name': short_name,
'version': '006',
'cycle': cycle,
'page_size': 100,
'page_num': 1,
'polygon': polygon,
}
    
granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

In [16]:
help(requests.get)

Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [30]:
help(requests.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) A JSON serializable Python object to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [59]:
polygon[

NameError: name 'geometry' is not defined

In [55]:
# Run post with geojson polygon
postP = {
    'shapefile' : polyPath,
    'type' : 'application/geo+json', 
    #'provider' : "PROV1",
}
polyPost = requests.post(granule_search_url, params=postP)

In [56]:
polyPost.content

b'<?xml version="1.0" encoding="UTF-8"?><errors><error>The CMR does not allow querying across granules in all collections. To help optimize your search, you should limit your query using conditions that identify one or more collections, such as provider, provider_id, concept_id, collection_concept_id, short_name, version or entry_title. For any questions please contact cmr-support@earthdata.nasa.gov.</error></errors>'

In [27]:

response = requests.get(granule_search_url, params=search_params, headers=headers)
result = json.loads(response.content)

In [28]:
result

In [10]:
granules = []
headers={'Accept': 'application/json'}
while True:
    response = requests.get(granule_search_url, params=search_params, headers=headers)
    result = json.loads(response.content)

    if len(result['feed']['entry']) == 0:
    #    # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    search_params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')

granule_sizes = [float(granule['granule_size']) for granule in granules]

print(f'The average size of each granule is {np.mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')

KeyError: 'feed'